In [1]:
from executorlib import SingleNodeExecutor

In [2]:
with SingleNodeExecutor() as exe:
    results = exe.map(sum, [[5, 5], [6, 6], [7, 7]])
    print(list(results))

[10, 12, 14]


In [3]:
from ase.build import bulk
import numpy as np
from atomistics.workflows import (
    analyse_results_for_elastic_matrix,
    get_tasks_for_elastic_matrix,
)
from atomistics.calculators import evaluate_with_lammpslib, get_potential_by_name

/Users/janssen/projects/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)


In [4]:
structure = bulk("Al", cubic=True)

In [5]:
df_pot_selected = get_potential_by_name(
    potential_name="1999--Mishin-Y--Al--LAMMPS--ipr1",
)

In [6]:
result_dict = evaluate_with_lammpslib(
    task_dict={"optimize_positions_and_volume": structure},
    potential_dataframe=df_pot_selected,
)
task_dict, sym_dict = get_tasks_for_elastic_matrix(
    structure=result_dict["structure_with_optimized_positions_and_volume"],
    eps_range=0.005,
    num_of_point=5,
    zero_strain_job_name="s_e_0",
    sqrt_eta=True,
)

In [7]:
task_dict['calc_energy'].keys()

odict_keys(['s_e_0', 's_01_e_m0_00500', 's_01_e_m0_00250', 's_01_e_0_00250', 's_01_e_0_00500', 's_08_e_m0_00500', 's_08_e_m0_00250', 's_08_e_0_00250', 's_08_e_0_00500', 's_23_e_m0_00500', 's_23_e_m0_00250', 's_23_e_0_00250', 's_23_e_0_00500'])

In [8]:
task_lst = []


In [9]:
with SingleNodeExecutor() as exe:
    future_lst = []
    for k, v in task_dict['calc_energy'].items():
        future_lst.append(exe.submit(evaluate_with_lammpslib, task_dict={'calc_energy': {k: v}}, potential_dataframe=df_pot_selected))
    results_lst = [f.result() for f in future_lst]

/Users/janssen/projects/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)
/Users/janssen/projects/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)
/Users/janssen/projects/atomistics/atomistics/calculators/__init__.py:63: UserWarning: calc_static_with_qe(), evaluate_with_qe() and optimize_positions_and_volume_with_qe() are not available as the import of the module named 'pwtools' failed.
  raise_warning(module_list=quantum_espresso_function, import_error=e)
/Users/janssen/projects/atomistics/atomistics/calculators/

In [10]:
result_dict = {}
for r in results_lst:
    result_dict.update(r["energy"])

In [11]:
result_dict

{'s_e_0': -13.439999952735112,
 's_01_e_m0_00500': -13.436320248980278,
 's_01_e_m0_00250': -13.439079680886989,
 's_01_e_0_00250': -13.439084974614394,
 's_01_e_0_00500': -13.436364320399795,
 's_08_e_m0_00500': -13.43817471490433,
 's_08_e_m0_00250': -13.439544638502628,
 's_08_e_0_00250': -13.43954822781134,
 's_08_e_0_00500': -13.438204192615176,
 's_23_e_m0_00500': -13.437971451918397,
 's_23_e_m0_00250': -13.439501038418321,
 's_23_e_0_00250': -13.439515785430656,
 's_23_e_0_00500': -13.438089441277944}

In [12]:
elastic_dict, sym_dict = analyse_results_for_elastic_matrix(
    output_dict={"energy": result_dict},
    sym_dict=sym_dict,
    fit_order=2,
    zero_strain_job_name="s_e_0",
)

In [13]:
elastic_dict['elastic_matrix']

array([[114.103117  ,  60.51102935,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935, 114.103117  ,  60.51102935,   0.        ,
          0.        ,   0.        ],
       [ 60.51102935,  60.51102935, 114.103117  ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,  31.67489592,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
         31.67489592,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,  31.67489592]])